In [106]:
# imports
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [107]:
# load dataset
df = pd.read_csv('/kaggle/input/wine-quality-dataset/WineQT.csv')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


In [108]:
# preprocess dataset
df = df.drop(['Id'], axis=1)

X = df.drop(['quality'], axis=1).values
y = df['quality'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [109]:
class WineNeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(WineNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        x = self.layer3(x)
        return x

In [110]:
# create dataset class
class WineDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [111]:
# training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

In [112]:
# create the model
train_dataset = WineDataset(X_train, y_train)
test_dataset = WineDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

input_size = X_train.shape[1]
model = WineNeuralNetwork(input_size)
criterion = nn.MSELoss()
optimizer= optim.AdamW(model.parameters(), lr=0.001)

train_model(model, train_loader, test_loader, criterion, optimizer, 40)

Epoch [1/40], Loss: 17.9353, Val Loss: 15.6427
Epoch [2/40], Loss: 4.8880, Val Loss: 6.1990
Epoch [3/40], Loss: 2.9637, Val Loss: 2.1637
Epoch [4/40], Loss: 1.1587, Val Loss: 0.8460
Epoch [5/40], Loss: 0.3420, Val Loss: 0.5807
Epoch [6/40], Loss: 0.7326, Val Loss: 0.5307
Epoch [7/40], Loss: 0.4901, Val Loss: 0.5142
Epoch [8/40], Loss: 0.1812, Val Loss: 0.4986
Epoch [9/40], Loss: 0.4680, Val Loss: 0.4851
Epoch [10/40], Loss: 0.4729, Val Loss: 0.4725
Epoch [11/40], Loss: 0.6865, Val Loss: 0.4614
Epoch [12/40], Loss: 0.4503, Val Loss: 0.4507
Epoch [13/40], Loss: 0.3632, Val Loss: 0.4419
Epoch [14/40], Loss: 0.3442, Val Loss: 0.4342
Epoch [15/40], Loss: 0.5753, Val Loss: 0.4287
Epoch [16/40], Loss: 0.4868, Val Loss: 0.4242
Epoch [17/40], Loss: 0.5427, Val Loss: 0.4202
Epoch [18/40], Loss: 0.5022, Val Loss: 0.4167
Epoch [19/40], Loss: 0.4428, Val Loss: 0.4138
Epoch [20/40], Loss: 0.2983, Val Loss: 0.4115
Epoch [21/40], Loss: 0.3308, Val Loss: 0.4097
Epoch [22/40], Loss: 0.4957, Val Loss: 0.

In [113]:
# Evaluate the model
model.eval()
test_loss = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        test_loss += criterion(outputs, labels).item()

print(f'Test Loss: {test_loss/len(test_loader):.4f}')

Test Loss: 0.3892
